# Example Notebook to showcasing how we interact with JTReaders

In [1]:
# First change dir to JTR parent
import os
os.chdir('..')

### Bookkeeping of all existing readers: `readers.py`

In [2]:
import jack.readers as readers

In [3]:
print("Existing models:\n%s" % ", ".join(readers.readers.keys()))

Existing models:
complex_reader, fastqa_reader, dam_snli_reader, distmult_reader, cbilstm_snli_reader, transe_reader, bidaf_reader, modelf_reader, esim_snli_reader


### Create a reader

In [4]:
%%script bash
bash data/GloVe/download_small.sh

glove.6B.50d.txt already exists! Doing nothing!


In [5]:
from jack.io.embeddings.embeddings import load_embeddings
from jack.util.vocab import Vocab

# we need a vocabulary (with embeddings for our fastqa_reader, but this is not always necessary)
embeddings = load_embeddings('data/GloVe/glove.6B.50d.txt', 'glove')
vocab = Vocab(emb=embeddings, init_from_embeddings=True)

# and a config
config = {"repr_dim": 10, "repr_dim_input": embeddings.lookup.shape[1], "model": "fastqa_reader", "max_span_size": 10}

In [6]:
# create example reader
from jack.core import SharedResources

svac = SharedResources(vocab, config)
fastqa_reader = readers.readers["fastqa_reader"](svac)

### Setting up a reader from training data 

In [7]:
from jack.io.load import load_jack
train_data = load_jack('data/SQuAD/snippet.jtr.json')
# all parameters are initialized after this call
fastqa_reader.setup_from_data(train_data)

### Saving the reader

In [8]:
fastqa_reader.store("/tmp/fastqa_reader")

In [9]:
%%sh
ls /tmp/fastqa_reader/

checkpoint
model_module.data-00000-of-00001
model_module.index
model_module.meta
shared_resources
shared_resources_vocab


### Loading the reader

In [10]:
# we can simply load a setup reader
fastqa_reader.load("/tmp/fastqa_reader")

INFO:tensorflow:Restoring parameters from /tmp/fastqa_reader/model_module


In [11]:
import tensorflow as tf
# reset graph -> computation graph is gone
tf.reset_default_graph()

# or setup a new reader from file 
svac = SharedResources()
fastqa_reader = readers.readers["fastqa_reader"](svac)
fastqa_reader.load_and_setup("/tmp/fastqa_reader")

INFO:tensorflow:Restoring parameters from /tmp/fastqa_reader/model_module


In [12]:
# reset graph -> computation graph is gone
tf.reset_default_graph()

# or even shorter, use the utility function for creating and loading a reader from file
fastqa_reader = readers.reader_from_file("/tmp/fastqa_reader")

INFO:tensorflow:Restoring parameters from /tmp/fastqa_reader/model_module


### Applying the reader

In [13]:
from jack.io.load import load_jack
data = load_jack('data/SQuAD/snippet.jtr.json')

# take a list of inputs, e.g., from our training data
questions = [q for q, a in data]

In [14]:
# Of course the output is not correct because the model was not trained at all
for q, a in zip(questions, fastqa_reader(questions)):
    print("Question: " + q.question)
    print("Answer:   %s \t %.3f" % (a[0].text, a[0].score))
    print()

Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Answer:   's gold dome is a golden 	 0.554

Question: What is in front of the Notre Dame Main Building?
Answer:   's gold dome is a golden 	 0.554

Question: The Basilica of the Sacred heart at Notre Dame is beside to which structure?
Answer:   ' 	 0.571

Question: What is the Grotto at Notre Dame?
Answer:   reflection 	 0.489

Question: What sits on top of the Main Building at Notre Dame?
Answer:   's gold dome is a golden 	 0.599

Question: When did the Scholastic Magazine of Notre dame begin publishing?
Answer:   ' 	 0.416

Question: How often is Notre Dame's the Juggler published?
Answer:   faculty advisor or any editorial 	 0.440

Question: What is the daily student paper at Notre Dame called?
Answer:   artwork. The Dome yearbook is published 	 0.533

Question: How many student news papers are found at Notre Dame?
Answer:   's College. 	 0.503

Question: In what year did the student paper Common Sens

# Training

In [15]:
# for training we use the bin/jack-train.py script, however, programatically we could quickly train a model like this
from jack.util.hooks import LossHook, ExamplesPerSecHook
import tensorflow as tf

# setup reader ini training mode
tf.reset_default_graph()
fastqa_reader = readers.readers["fastqa_reader"](svac)
fastqa_reader.setup_from_data(train_data, is_training=True)

batch_size=len(train_data)
hooks = [LossHook(fastqa_reader, iter_interval=1), 
         ExamplesPerSecHook(fastqa_reader, batch_size, iter_interval=1)]
optimizer = tf.train.AdamOptimizer(0.1)
fastqa_reader.train(optimizer, batch_size=batch_size, hooks=hooks, max_epochs=20, training_set=train_data)

/usr/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:jack.core.reader:Number of parameters: 6341
INFO:jack.core.reader:Start training...
INFO:jack.util.hooks:Epoch 1	Iter 1	train loss 10.044841766357422
INFO:jack.util.hooks:Epoch 2	Iter 2	train loss 9.546087265014648
INFO:jack.util.hooks:Epoch 3	Iter 3	train loss 8.032719612121582
INFO:jack.util.hooks:Epoch 4	Iter 4	train loss 7.163144588470459
INFO:jack.util.hooks:Epoch 5	Iter 5	train loss 7.233156681060791
INFO:jack.util.hooks:Epoch 6	Iter 6	train loss 6.6085524559021
INFO:jack.util.hooks:Epoch 7	Iter 7	train loss 5.957317352294922
INFO:jack.util.hooks:Epoch 8	Iter 8	train loss 5.243900775909424
INFO:jack.util.hooks:Epoch 9	Iter 9	train loss 4.88168478012085
INFO:jack.util.hooks:Epoch 10	Iter 10	train loss 4.474356651306152
INFO:jack.util.hooks:Epoch 11	Iter 11	train loss 3.6819164752960205
INFO:jack.util.hooks:Epoch 12	Iter 12	train loss 3.091872453689575
INFO:jack.util.hooks:Epoch 13	Iter 13	train loss 2.5408599376678467
INFO:jack.util.hooks:Epoch 14	Iter 14	train loss 2.1169764

In [16]:
# After training the output should look better
for q, a in zip(questions, fastqa_reader(questions)):
    print("Question: " + q.question)
    print("Answer:   %s \t %.3f" % (a[0].text, a[0].score))
    print()

Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Answer:   Saint Bernadette Soubirous 	 26.419

Question: What is in front of the Notre Dame Main Building?
Answer:   a golden statue of the Virgin Mary 	 21.271

Question: The Basilica of the Sacred heart at Notre Dame is beside to which structure?
Answer:   the Main Building 	 20.842

Question: What is the Grotto at Notre Dame?
Answer:   a Marian place of prayer and reflection 	 19.491

Question: What sits on top of the Main Building at Notre Dame?
Answer:   a golden statue of the Virgin Mary 	 21.536

Question: When did the Scholastic Magazine of Notre dame begin publishing?
Answer:   September 1876 	 31.663

Question: How often is Notre Dame's the Juggler published?
Answer:   twice 	 32.082

Question: What is the daily student paper at Notre Dame called?
Answer:   September 1876 	 28.987

Question: How many student news papers are found at Notre Dame?
Answer:   three 	 33.853

Question: In what year di